In [168]:
import pandas as pd
import os
from io import StringIO
import re
import numpy as np
import pandasql as ps

In [152]:
data = 'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'

<B>Reading the given string into a dataframe using StringIO command</B>

In [153]:
data1 = StringIO(data)

df = pd.read_csv(data1, sep=";")

df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],NaN,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",NaN,Ottawa_VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055.0,london_MONTreal


<B>Cleaning the Airline Code column</B>

In [154]:
def Airline_code_cleaner(x):
    return re.sub(r'[^a-zA-Z ]+', '', x).strip()

In [155]:
df['Airline Code'] = list(map(Airline_code_cleaner, df['Airline Code']))

In [156]:
df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada,"[21, 40]",20015.0,WAterLoo_NEWYork
1,Air France,[],NaN,Montreal_TORONTO
2,Porter Airways,"[60, 22, 87]",20035.0,CALgary_Ottawa
3,Air France,"[78, 66]",NaN,Ottawa_VANcouvER
4,Lufthansa,"[12, 33]",20055.0,london_MONTreal


<B>Cleaning the 'To_From' column and splitting it into 2 separate columns titled 'To' and 'From'</B>

In [157]:
def case_cleaner(x):
    return x.upper().strip()

In [158]:
df['To_From'] = list(map(lambda x : x.split('_'), df['To_From']))

In [159]:
df[['To','From']] = pd.DataFrame(df.To_From.tolist(), index= df.index)

In [160]:
df.drop('To_From', axis = 1, inplace = True)

In [161]:
df['To'] = list(map(case_cleaner, df['To']))
df['From'] = list(map(case_cleaner, df['From']))

In [163]:
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015.0,WATERLOO,NEWYORK
1,Air France,[],NaN,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035.0,CALGARY,OTTAWA
3,Air France,"[78, 66]",NaN,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055.0,LONDON,MONTREAL


<B>Cleaning the FlightCodes column, and recasting it as int type</B>

In [165]:
df['FlightCodes'] = df['FlightCodes'].interpolate().astype(int)

In [175]:
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL


### SQL QUERY FOR ALL FLIGHTS LEAVING FROM WATERLOO

In [186]:
query = "SELECT * FROM df WHERE [From] = 'WATERLOO'"

In [187]:
query = ps.sqldf(query)